In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator
from numpy.linalg import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import sqlite3 as sql

In [2]:
df = pd.read_csv('telco.csv')

In [3]:
num_cols = []
categorical_cols = []

In [4]:
def sort_cols(data):


    for i in data.columns:
        if data[i].dtypes == "object":
            categorical_cols.append(i)
        else :
            num_cols.append(i)

In [5]:
sort_cols(df)

In [6]:
def fix_cat_cols(data,cat_columns):
    for i in cat_columns:
        data[i] = data[i].fillna(data[i].mode()[0])

In [7]:
fix_cat_cols(df,categorical_cols)

In [8]:
def fix_num_cols(data,num_columns):
    for i in num_columns:
        data[i] = data[i].fillna(data[i].mean())

In [9]:
fix_num_cols(df,num_cols)

In [10]:
df['Total_volume (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

In [11]:
df['Session frequency'] = df.groupby(['Bearer Id'])['Dur. (ms)'].transform('count')

In [12]:
df['AVG RTT (ms)'] = df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']
df['Avg Bearer TP (kbps)'] = df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']
df['TCP Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']

In [13]:
needed_cols = ['Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)',"Dur. (ms)",'Total_volume (Bytes)']

In [14]:
for _ in needed_cols:
    Q1 = df[_].quantile(0.05)
    Q3 = df[_].quantile(0.95)

    df[_] = np.where(df[_] < Q1, df[_].mean(), df[_] )
    df[_] = np.where(df[_] > Q3, df[_].mean(), df[_] )
    print(df[_].skew())

0.751921435734779
0.8854286436014182
1.565286027050755
2.8244072963820197
-0.864194508047273
-1.0328094514997825
0.6229782068972313
-0.01794153047691228


In [15]:
X = df[['Session frequency','Dur. (ms)','Total_volume (Bytes)']]

In [16]:
df.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total_volume (Bytes),Session frequency,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
count,1.500010e+05,150001.000000,150001.000000,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,150001.000000,150001.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,150001.000000,150001.000000,1.500010e+05
mean,1.013887e+19,499.188200,498.800880,97258.895140,2.082016e+14,4.188282e+10,4.847455e+13,71.899207,11.479413,10223.694386,...,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08,4.967870e+08,7.779275,127.458589,15070.474573,2.156957e+07
std,2.883600e+18,288.610872,288.096693,45597.164746,2.144722e+10,2.438731e+12,2.237359e+13,39.227568,9.154229,15369.731190,...,4.782700e+06,2.432050e+08,4.769004e+06,1.127635e+07,2.441421e+08,2.084067e+08,80.185392,565.144762,26825.755093,1.184392e+08
min,6.917538e+18,0.000000,0.000000,19826.000000,2.040471e+14,3.360100e+10,4.400152e+11,23.000000,1.000000,11.000000,...,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06,1.158135e+08,1.000000,0.000000,0.000000,8.600000e+01
25%,7.349883e+18,250.000000,251.000000,72290.000000,2.082014e+14,3.365151e+10,3.546261e+13,37.000000,4.000000,44.000000,...,4.128476e+06,2.101869e+08,4.145943e+06,3.322203e+07,2.431072e+08,3.267124e+08,1.000000,40.000000,90.000000,1.492449e+06
50%,7.349883e+18,499.000000,500.000000,86400.000000,2.082015e+14,3.366380e+10,3.572241e+13,59.000000,9.000000,124.000000,...,8.291208e+06,4.218030e+08,8.267071e+06,4.114324e+07,4.558409e+08,4.968610e+08,1.000000,67.000000,125.000000,2.156957e+07
75%,1.304243e+19,749.000000,750.000000,115969.000000,2.082018e+14,3.368473e+10,8.611970e+13,109.795706,17.662883,13944.643396,...,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657051e+08,6.642172e+08,1.000000,127.458589,22348.000000,2.156957e+07
max,1.318654e+19,999.000000,999.000000,241538.000000,2.140743e+14,8.823971e+14,9.900120e+13,227.000000,44.000000,62544.000000,...,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08,8.753137e+08,991.000000,96924.000000,382262.000000,4.344116e+09


In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col_names = ["Session frequency","Dur. (ms)",'Total_volume (Bytes)']
X[col_names] = scaler.fit_transform(X[col_names])

C:\Users\CLASSISH\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [18]:
kmeans = KMeans(
    init= "random",
    n_clusters = 3,
    n_init= 10,
    max_iter= 300,
    random_state= 42
)

In [19]:
kmeans.fit(X)

KMeans(init='random', n_clusters=3, random_state=42)

In [20]:
X['Engagement_labels'] = kmeans.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/3504866275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Engagement_labels'] = kmeans.labels_


In [21]:
X

,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels
0,-0.084545,0.259323,-0.725304,0
1,-0.084545,0.259323,1.009560,0
2,-0.084545,0.259323,-0.907344,0
3,-0.084545,0.259323,0.091936,0
4,-0.084545,0.259323,0.532107,0
...,...,...,...,...
149996,-0.084545,-0.351534,0.647856,0
149997,-0.084545,0.015595,1.002835,0
149998,-0.072074,0.021714,0.628156,0
149999,-0.072074,0.014280,-0.457113,0


In [22]:
X.describe()

,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels
count,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000
mean,2.737940e-17,-1.250547e-16,3.718009e-16,0.252792
std,1.000003e+00,1.000003e+00,1.000003e+00,0.449558
min,-8.454529e-02,-1.698201e+00,-1.828035e+00,0.000000
25%,-8.454529e-02,-5.475994e-01,-8.160735e-01,0.000000
50%,-8.454529e-02,-2.381493e-01,3.548409e-04,0.000000
75%,-8.454529e-02,4.103362e-01,8.033842e-01,0.000000
max,1.226188e+01,3.164223e+00,1.816294e+00,2.000000


In [23]:
X1 = X[X.Engagement_labels==1]
X0 = X[X.Engagement_labels==0]
X2 = X[X.Engagement_labels==2]

In [24]:
needed_cols = ['Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)']

In [25]:
df.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total_volume (Bytes),Session frequency,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
count,1.500010e+05,150001.000000,150001.000000,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,150001.000000,150001.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,150001.000000,150001.000000,1.500010e+05
mean,1.013887e+19,499.188200,498.800880,97258.895140,2.082016e+14,4.188282e+10,4.847455e+13,71.899207,11.479413,10223.694386,...,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08,4.967870e+08,7.779275,127.458589,15070.474573,2.156957e+07
std,2.883600e+18,288.610872,288.096693,45597.164746,2.144722e+10,2.438731e+12,2.237359e+13,39.227568,9.154229,15369.731190,...,4.782700e+06,2.432050e+08,4.769004e+06,1.127635e+07,2.441421e+08,2.084067e+08,80.185392,565.144762,26825.755093,1.184392e+08
min,6.917538e+18,0.000000,0.000000,19826.000000,2.040471e+14,3.360100e+10,4.400152e+11,23.000000,1.000000,11.000000,...,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06,1.158135e+08,1.000000,0.000000,0.000000,8.600000e+01
25%,7.349883e+18,250.000000,251.000000,72290.000000,2.082014e+14,3.365151e+10,3.546261e+13,37.000000,4.000000,44.000000,...,4.128476e+06,2.101869e+08,4.145943e+06,3.322203e+07,2.431072e+08,3.267124e+08,1.000000,40.000000,90.000000,1.492449e+06
50%,7.349883e+18,499.000000,500.000000,86400.000000,2.082015e+14,3.366380e+10,3.572241e+13,59.000000,9.000000,124.000000,...,8.291208e+06,4.218030e+08,8.267071e+06,4.114324e+07,4.558409e+08,4.968610e+08,1.000000,67.000000,125.000000,2.156957e+07
75%,1.304243e+19,749.000000,750.000000,115969.000000,2.082018e+14,3.368473e+10,8.611970e+13,109.795706,17.662883,13944.643396,...,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657051e+08,6.642172e+08,1.000000,127.458589,22348.000000,2.156957e+07
max,1.318654e+19,999.000000,999.000000,241538.000000,2.140743e+14,8.823971e+14,9.900120e+13,227.000000,44.000000,62544.000000,...,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08,8.753137e+08,991.000000,96924.000000,382262.000000,4.344116e+09


In [26]:
scaler = StandardScaler()
cols_scaled = ['AVG RTT (ms)','Avg Bearer TP (kbps)','TCP Retrans. Vol (Bytes)']

In [27]:
df[cols_scaled] = scaler.fit_transform(df[cols_scaled])


In [28]:
Y = df[['AVG RTT (ms)','Avg Bearer TP (kbps)','TCP Retrans. Vol (Bytes)']]

In [29]:
Y

,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
0,-1.423686e-01,-5.592955e-01,0.0
1,-1.016709e-01,-5.602275e-01,0.0
2,5.029119e-17,-5.612340e-01,0.0
3,5.029119e-17,-5.585127e-01,0.0
4,5.029119e-17,-5.612340e-01,0.0
...,...,...,...
149996,-1.689105e-01,-5.574316e-01,0.0
149997,-1.742189e-01,-5.589227e-01,0.0
149998,-1.388297e-01,-5.584381e-01,0.0
149999,-1.512159e-01,-5.591464e-01,0.0


In [30]:
kmeans.fit(Y)

KMeans(init='random', n_clusters=3, random_state=42)

In [31]:
Y['Experience_labels'] = kmeans.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/1125441398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Experience_labels'] = kmeans.labels_


In [32]:
Y['Experience_labels'].value_counts()

1    121932
2     27813
0       256
Name: Experience_labels, dtype: int64

In [33]:
Y1 = Y[Y.Experience_labels==1]
Y0 = Y[Y.Experience_labels==0]
Y2 = Y[Y.Experience_labels==2]

In [34]:
X['Engagement_score'] =np.sqrt(((X2['Session frequency'] + X['Dur. (ms)'] + X['Total_volume (Bytes)']).pow(2)) - ((2**2)))
X['Engagement_score'] = X['Engagement_score'].fillna(0)                            

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/2325902614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Engagement_score'] =np.sqrt(((X2['Session frequency'] + X['Dur. (ms)'] + X['Total_volume (Bytes)']).pow(2)) - ((2**2)))
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/2325902614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Engagement_score'] = X['Engagement_score'].fillna(0)


In [35]:
X['Engagement_score'].value_counts()

0.000000     149010
12.256053        17
12.354166         8
9.709512          1
10.533272         1
              ...  
12.676896         1
10.599861         1
12.125485         1
11.065068         1
11.771246         1
Name: Engagement_score, Length: 969, dtype: int64

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Session frequency     150001 non-null  float64
 1   Dur. (ms)             150001 non-null  float64
 2   Total_volume (Bytes)  150001 non-null  float64
 3   Engagement_labels     150001 non-null  int32  
 4   Engagement_score      150001 non-null  float64
dtypes: float64(4), int32(1)
memory usage: 5.1 MB


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 60 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150001 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150001 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        150001 non-null  object 
 10  Avg RTT DL (ms)     

In [38]:
Y['Experience_labels'].value_counts()

1    121932
2     27813
0       256
Name: Experience_labels, dtype: int64

In [39]:
Y['Experience_score'] =np.sqrt(((Y2['AVG RTT (ms)'] + Y2['Avg Bearer TP (kbps)'] + Y2['TCP Retrans. Vol (Bytes)']).pow(2)) - ((2**2)))
Y['Experience_score'] = Y['Experience_score'].fillna(0)

C:\Users\CLASSISH\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/158759607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Experience_score'] =np.sqrt(((Y2['AVG RTT (ms)'] + Y2['Avg Bearer TP (kbps)'] + Y2['TCP Retrans. Vol (Bytes)']).pow(2)) - ((2**2)))
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/158759607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [40]:
Y

,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score
0,-1.423686e-01,-5.592955e-01,0.0,1,0.0
1,-1.016709e-01,-5.602275e-01,0.0,1,0.0
2,5.029119e-17,-5.612340e-01,0.0,1,0.0
3,5.029119e-17,-5.585127e-01,0.0,1,0.0
4,5.029119e-17,-5.612340e-01,0.0,1,0.0
...,...,...,...,...,...
149996,-1.689105e-01,-5.574316e-01,0.0,1,0.0
149997,-1.742189e-01,-5.589227e-01,0.0,1,0.0
149998,-1.388297e-01,-5.584381e-01,0.0,1,0.0
149999,-1.512159e-01,-5.591464e-01,0.0,1,0.0


In [41]:
Y['Experience_score'].value_counts()

0.000000    142515
2.664437         1
2.375491         1
1.049125         1
4.183476         1
             ...  
4.207123         1
2.594879         1
1.602872         1
2.420695         1
0.809304         1
Name: Experience_score, Length: 7487, dtype: int64

In [42]:
new_df= pd.DataFrame()

In [43]:
new_df['MSISDN/Number'] = df['MSISDN/Number']

In [44]:
new_df = new_df.join(X)
new_df = new_df.join(Y)

In [45]:
new_df['Experience_score'].value_counts()

0.000000    142515
2.664437         1
2.375491         1
1.049125         1
4.183476         1
             ...  
4.207123         1
2.594879         1
1.602872         1
2.420695         1
0.809304         1
Name: Experience_score, Length: 7487, dtype: int64

In [46]:
new_df['Satisfaction_score'] = ((new_df['Engagement_score'] + new_df['Experience_score'])/2)+1
new_df['Satisfaction_score'] = new_df['Satisfaction_score'].fillna(0)

In [47]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   MSISDN/Number             150001 non-null  float64
 1   Session frequency         150001 non-null  float64
 2   Dur. (ms)                 150001 non-null  float64
 3   Total_volume (Bytes)      150001 non-null  float64
 4   Engagement_labels         150001 non-null  int32  
 5   Engagement_score          150001 non-null  float64
 6   AVG RTT (ms)              150001 non-null  float64
 7   Avg Bearer TP (kbps)      150001 non-null  float64
 8   TCP Retrans. Vol (Bytes)  150001 non-null  float64
 9   Experience_labels         150001 non-null  int32  
 10  Experience_score          150001 non-null  float64
 11  Satisfaction_score        150001 non-null  float64
dtypes: float64(10), int32(2)
memory usage: 12.6 MB


In [48]:
new_df

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
0,3.366496e+10,-0.084545,0.259323,-0.725304,0,0.000000,-1.423686e-01,-5.592955e-01,0.0,1,0.0,1.000000
1,3.368185e+10,-0.084545,0.259323,1.009560,0,0.000000,-1.016709e-01,-5.602275e-01,0.0,1,0.0,1.000000
2,3.376063e+10,-0.084545,0.259323,-0.907344,0,0.000000,5.029119e-17,-5.612340e-01,0.0,1,0.0,1.000000
3,3.375034e+10,-0.084545,0.259323,0.091936,0,0.000000,5.029119e-17,-5.585127e-01,0.0,1,0.0,1.000000
4,3.369980e+10,-0.084545,0.259323,0.532107,0,0.000000,5.029119e-17,-5.612340e-01,0.0,1,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
149996,3.365069e+10,-0.084545,-0.351534,0.647856,0,0.000000,-1.689105e-01,-5.574316e-01,0.0,1,0.0,1.000000
149997,3.366345e+10,-0.084545,0.015595,1.002835,0,0.000000,-1.742189e-01,-5.589227e-01,0.0,1,0.0,1.000000
149998,3.362189e+10,-0.072074,0.021714,0.628156,0,0.000000,-1.388297e-01,-5.584381e-01,0.0,1,0.0,1.000000
149999,3.361962e+10,-0.072074,0.014280,-0.457113,0,0.000000,-1.512159e-01,-5.591464e-01,0.0,1,0.0,1.000000


In [49]:
new_df['Satisfaction_score'].sort_values(ascending=False)[:10]

97480     16.749244
43628     11.571911
65019     11.416402
86703      9.661529
20967      9.377676
102536     9.063403
98129      9.048396
102961     9.039821
27179      9.011007
27699      8.905858
Name: Satisfaction_score, dtype: float64

In [50]:
new_df.sort_values(ascending=False, by='Satisfaction_score')[:10]

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
97480,3.365129e+10,-0.084545,-1.410287,-1.821990,0,0.000000,2.909626e+01,2.518988,-0.053324,2,31.498488,16.749244
43628,3.366540e+10,-0.084545,-0.632144,1.553055,0,0.000000,1.853255e+01,2.882297,-0.176650,2,21.143822,11.571911
65019,4.188282e+10,12.261884,-0.238347,-1.588531,2,10.241551,-1.034404e-01,3.376786,7.505089,2,10.591254,11.416402
86703,4.188282e+10,12.261884,-1.186896,0.858076,2,11.764269,1.212816e-01,3.762200,2.024151,2,5.558788,9.661529
20967,3.367677e+10,12.261884,0.161187,0.964286,2,13.237120,-1.512159e-01,4.357973,-0.159787,2,3.518233,9.377676
102536,4.188282e+10,12.261884,2.331493,1.656973,2,16.126806,5.029119e-17,-0.560265,0.000000,1,0.000000,9.063403
98129,3.365831e+10,-0.072074,-1.336774,-0.004906,0,0.000000,1.372315e+01,2.659450,-0.162038,2,16.096792,9.048396
102961,4.188282e+10,12.261884,2.454746,1.486915,2,16.079643,5.029119e-17,-0.557693,0.000000,1,0.000000,9.039821
27179,4.188282e+10,12.261884,2.592080,1.292395,2,16.022014,5.029119e-17,-0.559706,0.000000,1,0.000000,9.011007
27699,4.188282e+10,12.261884,2.055663,1.620154,2,15.811715,5.029119e-17,-0.561234,0.000000,1,0.000000,8.905858


In [51]:
y = new_df['Satisfaction_score']

In [52]:
cols_drop = ['MSISDN/Number','Satisfaction_score','Engagement_labels','Experience_labels','Experience_score','Engagement_score']
X = new_df.drop(cols_drop,axis=1)

In [53]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Session frequency         150001 non-null  float64
 1   Dur. (ms)                 150001 non-null  float64
 2   Total_volume (Bytes)      150001 non-null  float64
 3   AVG RTT (ms)              150001 non-null  float64
 4   Avg Bearer TP (kbps)      150001 non-null  float64
 5   TCP Retrans. Vol (Bytes)  150001 non-null  float64
dtypes: float64(6)
memory usage: 6.9 MB


In [54]:
y.value_counts()

1.000000    141548
7.128027        17
7.177083         8
6.507902         1
2.025818         1
             ...  
1.569167         1
1.524939         1
1.567662         1
1.490157         1
1.404652         1
Name: Satisfaction_score, Length: 8431, dtype: int64

In [55]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, test_size= 0.25)

In [56]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112500, 6)
(37501, 6)
(112500,)
(37501,)


In [57]:
lin_reg = LinearRegression()


In [58]:
lin_reg.fit(X_train,y_train)
print(lin_reg.intercept_)
print(lin_reg.coef_)

1.0973035371311524
[0.46391373 0.02832734 0.00234937 0.00808562 0.2134049  0.02917658]


In [59]:
model = lin_reg.predict(X_test)

In [60]:
print(f"Root mean squared error {np.sqrt(mean_squared_error(y_test,model))}")
print(f"Mean squared error {mean_squared_error(y_test,model)}")
print(f"Mean Absolute error {mean_absolute_error(y_test,model)}")

Root mean squared error 0.2587582968339095
Mean squared error 0.06695585618038562
Mean Absolute error 0.12442958803583046


In [61]:
score_check = pd.DataFrame()

In [62]:
score_check['y_test'] = y_test

In [63]:
score_check['model'] = model

In [64]:
score_check

,y_test,model
16311,1.000000,1.144851
102956,1.000000,1.015135
21093,7.264359,6.829464
48936,1.000000,1.089237
9774,1.000000,1.072704
...,...,...
4451,1.000000,1.224688
117051,1.000000,1.334689
127975,1.000000,0.993286
87368,1.000000,0.903421


In [65]:
metrics = new_df[['Engagement_score','Experience_score']]

In [66]:
met_cols = ['Engagement_score','Experience_score']

In [67]:
metrics[met_cols] = scaler.fit_transform(metrics[met_cols])

C:\Users\CLASSISH\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [68]:
Kmeans_2 = KMeans(
    init= "random",
    n_clusters = 2,
    n_init= 10,
    max_iter= 300,
    random_state= 42
)

In [69]:
Kmeans_2.fit(metrics)

KMeans(init='random', n_clusters=2, random_state=42)

In [70]:
metrics

,Engagement_score,Experience_score
0,-0.080986,-0.179166
1,-0.080986,-0.179166
2,-0.080986,-0.179166
3,-0.080986,-0.179166
4,-0.080986,-0.179166
...,...,...
149996,-0.080986,-0.179166
149997,-0.080986,-0.179166
149998,-0.080986,-0.179166
149999,-0.080986,-0.179166


In [71]:
metrics['cluster'] = Kmeans_2.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/2972770247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['cluster'] = Kmeans_2.labels_


In [72]:
metrics['Satisfaction_score'] = new_df['Satisfaction_score']

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_4104/3150825679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['Satisfaction_score'] = new_df['Satisfaction_score']


In [73]:
satisfied_per_cluster = metrics.groupby('cluster').agg({'Satisfaction_score':["median","mean"]})

In [74]:
satisfied_per_cluster

Satisfaction_score          
                    median      mean
cluster                             
0                 1.000000  1.050026
1                 2.387557  2.693039

In [75]:
experience_per_cluster = metrics.groupby('cluster').agg({'Experience_score':["median","mean"]})


In [76]:
experience_per_cluster

Experience_score          
                  median      mean
cluster                           
0              -0.179166 -0.144973
1               4.043916  4.931155

In [77]:
new_df

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
0,3.366496e+10,-0.084545,0.259323,-0.725304,0,0.000000,-1.423686e-01,-5.592955e-01,0.0,1,0.0,1.000000
1,3.368185e+10,-0.084545,0.259323,1.009560,0,0.000000,-1.016709e-01,-5.602275e-01,0.0,1,0.0,1.000000
2,3.376063e+10,-0.084545,0.259323,-0.907344,0,0.000000,5.029119e-17,-5.612340e-01,0.0,1,0.0,1.000000
3,3.375034e+10,-0.084545,0.259323,0.091936,0,0.000000,5.029119e-17,-5.585127e-01,0.0,1,0.0,1.000000
4,3.369980e+10,-0.084545,0.259323,0.532107,0,0.000000,5.029119e-17,-5.612340e-01,0.0,1,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
149996,3.365069e+10,-0.084545,-0.351534,0.647856,0,0.000000,-1.689105e-01,-5.574316e-01,0.0,1,0.0,1.000000
149997,3.366345e+10,-0.084545,0.015595,1.002835,0,0.000000,-1.742189e-01,-5.589227e-01,0.0,1,0.0,1.000000
149998,3.362189e+10,-0.072074,0.021714,0.628156,0,0.000000,-1.388297e-01,-5.584381e-01,0.0,1,0.0,1.000000
149999,3.361962e+10,-0.072074,0.014280,-0.457113,0,0.000000,-1.512159e-01,-5.591464e-01,0.0,1,0.0,1.000000


In [78]:
metrics

,Engagement_score,Experience_score,cluster,Satisfaction_score
0,-0.080986,-0.179166,0,1.000000
1,-0.080986,-0.179166,0,1.000000
2,-0.080986,-0.179166,0,1.000000
3,-0.080986,-0.179166,0,1.000000
4,-0.080986,-0.179166,0,1.000000
...,...,...,...,...
149996,-0.080986,-0.179166,0,1.000000
149997,-0.080986,-0.179166,0,1.000000
149998,-0.080986,-0.179166,0,1.000000
149999,-0.080986,-0.179166,0,1.000000


In [79]:
sql_df = pd.DataFrame()

In [80]:
sql_df['MSISDN/Number'] = new_df['MSISDN/Number']

In [81]:
cols_sql_df = ['Engagement_score','Experience_score','Satisfaction_score']
sql_df[cols_sql_df] = metrics[cols_sql_df]

In [82]:
sql_df

,MSISDN/Number,Engagement_score,Experience_score,Satisfaction_score
0,3.366496e+10,-0.080986,-0.179166,1.000000
1,3.368185e+10,-0.080986,-0.179166,1.000000
2,3.376063e+10,-0.080986,-0.179166,1.000000
3,3.375034e+10,-0.080986,-0.179166,1.000000
4,3.369980e+10,-0.080986,-0.179166,1.000000
...,...,...,...,...
149996,3.365069e+10,-0.080986,-0.179166,1.000000
149997,3.366345e+10,-0.080986,-0.179166,1.000000
149998,3.362189e+10,-0.080986,-0.179166,1.000000
149999,3.361962e+10,-0.080986,-0.179166,1.000000


In [83]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://',echo = False)

In [84]:
#conn = sql.connect('sql_df.db')
#sql_df.to_sql('sql_df', conn)

In [85]:
conn = sql.connect('sql_df.db')
sql_db = pd.read_sql('SELECT * FROM sql_df WHERE Satisfaction_score == 1', conn)

In [86]:
sql_db

,index,MSISDN/Number,Engagement_score,Experience_score,Satisfaction_score
0,63,3.369943e+10,-0.228068,-0.276332,1.0
1,64,3.365951e+10,-0.228068,-0.276332,1.0
2,67,3.365079e+10,-0.228068,-0.276332,1.0
3,70,3.368243e+10,-0.228068,-0.276332,1.0
4,73,3.368451e+10,-0.228068,-0.276332,1.0
...,...,...,...,...,...
114916,149994,3.364566e+10,-0.228068,-0.276332,1.0
114917,149996,3.365069e+10,-0.228068,-0.276332,1.0
114918,149997,3.366345e+10,-0.228068,-0.276332,1.0
114919,149998,3.362189e+10,-0.228068,-0.276332,1.0


In [87]:
import pickle
with open('model_pkl_1', 'wb') as files:
    pickle.dump(lin_reg, files)

In [88]:
import joblib

In [89]:
joblib.dump(lin_reg, 'model_joblib')

['model_joblib']